In [1]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, confusion_matrix ,recall_score, roc_auc_score , roc_curve, classification_report,ConfusionMatrixDisplay
tqdm.pandas()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# importing the Dataset

messages = pd.read_csv('SMSSpamCollection.txt', sep='\t', header=None, names=["label", "message"])

### Word 2 Vec Implementation

In [3]:
messages.head(1)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."


In [4]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()

corpus = []
for i in range(len(messages['message'])):
        review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
        review = review.lower()
        review = review.split()
        review = [lemmatizer.lemmatize(word) for word in review if word not in stopwords.words("english")]
        review = ' '.join(review)
        corpus.append(review)

In [5]:
corpus[:5]

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though']

In [6]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [7]:
words =[]
for sent in corpus:
    sent_token = sent_tokenize(sent)   ### convert every sentence in a list
    for sent in sent_token:
        ## Lowering every word
        words.append(simple_preprocess(sent))

In [8]:
words[1]

['ok', 'lar', 'joking', 'wif', 'oni']

In [9]:
import gensim

In [10]:
## LET'S TRAIN WORD2VEC FROM SCRATCH
model = gensim.models.Word2Vec(words, window = 5, min_count = 2)

### window :-> How Many Context Of The Word On The Left Hand Side And The Right Hand Side Of the That We Basically Have To Take

In [11]:
# model.wv.index_to_key

In [12]:
model.corpus_count  ## Total Vocaulary Size 

5564

In [13]:
model.epochs

5

In [14]:
##### We Are Getting Similar Word From Our Dataset

In [15]:
model.wv.similar_by_word('love')

[('day', 0.9997518658638),
 ('life', 0.999741792678833),
 ('hope', 0.9997262358665466),
 ('much', 0.9997085928916931),
 ('say', 0.9997046589851379),
 ('amp', 0.9996938705444336),
 ('one', 0.9996938705444336),
 ('thing', 0.9996882081031799),
 ('keep', 0.9996833801269531),
 ('need', 0.9996823668479919)]

In [16]:
model.wv.similar_by_word('happy')

[('year', 0.999449610710144),
 ('day', 0.9994052648544312),
 ('make', 0.9993911981582642),
 ('hello', 0.9993892908096313),
 ('dear', 0.9993500709533691),
 ('amp', 0.9993240833282471),
 ('new', 0.9993232488632202),
 ('keep', 0.9993211627006531),
 ('like', 0.9993187785148621),
 ('dont', 0.9993124604225159)]

In [17]:
model.wv['kid'].shape

(100,)

In [18]:
# words[0][0]

In [19]:
# model.wv[words[0][0]]

In [20]:
def avg_word2Vec(doc):
    #remove out of vocabulary words
#     sent = [word for word in doc if word in model.wv.index.to_key]
#     print(sent)
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis =0)
            ## or [np.zeros(len(model.wv.index.to_key))], axis =0  model.wv.index_to_key

In [21]:
avg_word2Vec(words[0][0])

C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

In [22]:
type(model.wv.index_to_key)

list

In [23]:
### Apply For The Entire Sentence 
X = []
for i in tqdm(range(len(words))):
#     print(words[i])
    X.append(avg_word2Vec(words[i]))

100%|████████████████████████████████████████████████████████████████████████████| 5564/5564 [00:01<00:00, 3209.15it/s]


In [24]:
type(X)

list

In [25]:
X_new = np.array(X, dtype=object)

In [26]:
X_new[0]

array([-0.10233888,  0.29385707,  0.14027993,  0.03930686,  0.03297272,
       -0.3215437 ,  0.07861059,  0.5157609 , -0.1601534 , -0.14989261,
       -0.1575335 , -0.3442289 , -0.0023139 ,  0.09813994,  0.07422768,
       -0.28780895,  0.01998659, -0.34101233, -0.02095743, -0.43834153,
        0.07106078,  0.1538647 ,  0.09947666, -0.12236571, -0.11105856,
        0.02325915, -0.21110852, -0.14859745, -0.22451498,  0.05217688,
        0.2840272 ,  0.06031932,  0.14916116, -0.23889585, -0.11094094,
        0.2570959 ,  0.03486114, -0.19416146, -0.17069827, -0.36665115,
        0.06519165, -0.22811462, -0.10086148,  0.05077912,  0.24369444,
       -0.13582043, -0.17591332,  0.0377537 ,  0.14573315,  0.23295414,
        0.17610247, -0.283479  , -0.04915407, -0.02367802, -0.1312682 ,
        0.20474158,  0.16993469, -0.0405581 , -0.27021635,  0.06982597,
        0.10879947,  0.12083304, -0.13319862, -0.00122989, -0.25690943,
        0.1571313 ,  0.08550588,  0.1942152 , -0.2783275 ,  0.32